## Secret Santa Emailer 🎅

Ho ho ho! Welcome to the Secret Santa Mailer! This jolly Python script magically pulls names from Santa's hat and sends delightful emails to each Secret Santa, revealing their lucky gift recipient. With the help of pandas, smtplib, and a sprinkle of holiday cheer, it ensures that no one knows who their Secret Santa is... not even you! So, get ready to spread the joy and have a Merry Christmas! 🎄✨

In [ ]:
import pandas as pd
import random
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 587
SENDER_EMAIL = "your-email@gmail.com"  
SENDER_PASSWORD = "app password"  # Replace with your app password (different from account PW)

## Loading your participants data

In [ ]:
file_path = "Secret Santa Participants List.xlsx" # Change this to the path of your Excel file
data = pd.read_excel(file_path)

data.columns = ["Name", "Email", "Preferences"]
data = data.dropna(subset=["Name", "Email"]).reset_index(drop=True)

## Assign Secret Santa Names

In [ ]:
def assign_secret_santa(names):
    assignments = names[:]
    while any(a == b for a, b in zip(assignments, names)):
        random.shuffle(assignments)
    return assignments

names = data["Name"].tolist()
emails = data["Email"].tolist()
preferences = data["Preferences"].tolist()
data["Preferences"] = data["Preferences"].fillna("")

assignments = assign_secret_santa(names)

pairs = {
    "Santa": names,
    "Santa_Email": emails,
    "Recipient": assignments,
    "Recipient_Preferences": [preferences[names.index(rec)] if rec in names else "" for rec in assignments]
}

pairs_df = pd.DataFrame(pairs)
# display(pairs_df) <- Uncomment this line to display the assignments in the console (to check for quality)
pairs_df.to_excel("SecretSanta_Assignments_SECRET.xlsx", index=False) # This file will store the final assignments - keep it secret! or 

## Prepare Email text

In [ ]:
def send_email(santa_name, santa_email, recipient_name, recipient_preferences, test_mode= False):
    # Adjust the below email text as you want!
    subject = "Your Secret Santa Assignment!"
    body = f"""
    Dear {santa_name},

    🎅 Thank you for participating in Secret Santa! 🎁
    
    Your recipient is: {recipient_name}
    Preferences: {recipient_preferences if recipient_preferences else "No specific preferences provided"}
    
    Happy gifting!

    Best,

    Your Secret Santa Bot
    """

    msg = MIMEMultipart()
    msg["From"] = SENDER_EMAIL
    msg["To"] = santa_email if not test_mode else SENDER_EMAIL  # Send to sender in test mode
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as server:
        server.ehlo()
        server.starttls()
        server.ehlo()
        server.login(SENDER_EMAIL, SENDER_PASSWORD)
        server.sendmail(SENDER_EMAIL, msg["To"], msg.as_string())

    print(f"Email sent to {'test recipient' if test_mode else santa_email}.")

## Make sure everything works correctly

Send a brief test email to your owner (sender account) to make sure it works

In [ ]:
send_email("test sender", SENDER_EMAIL, "test recipient", "test preferences", test_mode=True) 

## Final Step: Send out Emails

Uncomment the lines below to send the E-mail to every santa!

In [ ]:
# for santa, santa_email, recipient, recipient_preferences in zip(
#     pairs["Santa"], pairs["Santa_Email"], pairs["Recipient"], pairs["Recipient_Preferences"]
# ):
#     send_email(santa, santa_email, recipient, recipient_preferences)